In [ ]:
import tweepy
from tweepy import OAuthHandler
import re
import pandas as pd
import numpy as np
import json
import sys

In [ ]:
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''

nfinal = []

In [ ]:
#Fetch Tweets
def get_all_tweets(screen_name):

#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	for t in new_tweets:
		nfinal.append(t._json)

	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1

In [ ]:
#read the json file and create a dataframe containing text, date and time, no. of likes, no. of RT, no. of images
def parse_and_print_json(json_file_name):

        message, created_at, favorite_count, retweet_count, image_count=[],[],[],[],[]

        with open(json_file_name, 'r') as data_file:
            json_data = data_file.read()

        tweets = json.loads(json_data)

        for tweet in tweets:
            message.append(tweet["text"])
            created_at.append(tweet["created_at"])
            favorite_count.append(tweet["favorite_count"])
            retweet_count.append(tweet["retweet_count"])
            if "extended_entities" in tweet:
                image_count.append(len(tweet["extended_entities"]["media"]))
            else:
                image_count.append(0)

        df=pd.DataFrame({'Tweet text':message,
                        'Created At':created_at,
                        'Tweet Favorite Count':favorite_count,
                        'Retweet count':retweet_count,
                        'No. of images':image_count
                        })

        print(df)

In [ ]:
if __name__ == '__main__':
	#pass in the username of the account you want to download
	get_all_tweets("midasIIITD")
	with open('json_file.json', "w") as file_write:
		json.dump(nfinal, file_write)

	parse_and_print_json('json_file.json')